**2025 Military AI CONtest**<br>
AI Autonomous Driving – Pre-Training Code Sample<br> 
NVIDIA Jetson Nano/Wingbot/JupyterLab<br>
Version : v1.0<br>
File : robot_camera_aruco.ipynb


In [ ]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from collections import deque

# Jetson Nano 카메라 파이프라인
pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

if not cap.isOpened():
    print("Camera Open Error")
    exit()

video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='640px', height='480px'))
display(video_widget)

def frame_to_bytes(frame):
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

# ArUco 설정
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
print("[INFO] ArUco 설정 완료")

try:
    detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)
except:
    detector = None

log_buffer = deque(maxlen=5)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("[WARN] 프레임 읽기 실패")
            break

        frame = cv2.flip(frame, -1)
        if detector:
            corners, ids, _ = detector.detectMarkers(frame)
        else:
            corners, ids, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)
        if ids is not None:
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)
            detected_ids = [int(i[0]) for i in ids]
            current_time = time.strftime("%H:%M:%S")
            log_buffer.append(f"[{current_time}] 감지된 ArUco 마커: {detected_ids}")

            clear_output(wait=True)
            display(video_widget)
            for line in log_buffer:
                print(line)

        video_widget.value = frame_to_bytes(frame)

except KeyboardInterrupt:
    print("\n[EXIT] 카메라 스트림 종료")

finally:
    cap.release()
    print("[INFO] 리소스 해제 완료")


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x14\x0e\x0…